In [1]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# load data set
wbcdfile = "C:/Users/joeba/OneDrive/Documents/ML/wbcp/wbcp30.csv"
dataset = pd.read_csv(wbcdfile)

In [23]:
X = dataset.iloc[:, 3:35].values
x = dataset.iloc[:, 3:35].columns
Y = dataset.iloc[:, 1].values
y = ['N', 'R']

x

Index(['radius_m', 'texture_m', 'perimeter_m', 'area_m', 'smoothness_m',
       'compactness_ m', 'concavity_m', 'concave_[ponts_m', 'symmetry_m',
       'fractal_dimension_m', 'radius_s', 'texture_s', 'perimeter_s', 'area_s',
       'smoothness_s', 'compactness_ m.1', 'concavity_s', 'concave_[ponts_s',
       'symmetry_s', 'fractal_dimension_s', 'radius_w', 'texture_w',
       'perimeter_w', 'area_w', 'smoothness_w', 'compactness_ m.2',
       'concavity_w', 'concave_[ponts_w', 'symmetry_w', 'fractal_dimension_w',
       'tumor_size', 'lymph_node_status'],
      dtype='object')

In [14]:
# impute missing lymph node values
pd.isna(X)

#from sklearn.impute import KNNImputer
from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=2, weights='distance')
X = knn_imputer.fit_transform(X)

### simple imputer with a median strategy
#from sklearn.impute import SimpleImputer
#imputer = SimpleImputer(strategy='median', verbose=1)
#X = imputer.fit_transform(X)

X

array([[ 0.16123689,  1.11587983,  0.12067323, ..., -0.23228148,
         1.25      ,  1.        ],
       [ 0.15461071, -2.16881259,  0.28898063, ...,  1.29085593,
         0.25      ,  0.25      ],
       [ 0.90115958, -0.82212685,  0.75579549, ...,  0.15243472,
         0.        , -0.25      ],
       ...,
       [-0.02650469,  1.41917024, -0.11749762, ..., -1.07107571,
         0.6       , -0.25      ],
       [ 0.9122032 ,  0.20791607,  0.99396634, ...,  0.11775381,
         0.25      ,  0.        ],
       [-0.13031474,  1.21697663, -0.10797078, ..., -0.263333  ,
         0.5       , -0.25      ]])

In [15]:
# scale the data
from sklearn import preprocessing
#X = preprocessing.scale(X)
X = preprocessing.robust_scale(X)

In [16]:
# convert the 'N' and 'R' to 0 and 1 respectively for Y
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y2 = le.fit_transform(Y)
y2

array([0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [17]:
# training test set split - 75% being used to train and ensuring even stratification of output in test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.25, random_state=1, stratify=None)
print(len(y_test))

50


In [18]:
from sklearn import svm
svc = svm.SVC(gamma=0.15, kernel='rbf')
svc.fit(X_test, y_test)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.15, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [19]:
# evaluate the performance of the neural network
y_pred = svc.predict(X_test)
y_pred = np.rint(y_pred)
y_pred

array([0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.])

In [20]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

#            predicted
#            N   P
#           +-----
# actual  N |TN  FP
#         P |FN  TP

array([[36,  0],
       [ 2, 12]], dtype=int64)

In [21]:
# what are the results?
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
print('Accuracy : %.3f' % accuracy_score(y_pred, y_test))
print('Error    : %.3f' % (1 - accuracy_score(y_pred, y_test)))
print('Precision: %.3f' % precision_score(y_pred, y_test))
print('Recall   : %.3f' % recall_score(y_pred, y_test))

Accuracy : 0.960
Error    : 0.040
Precision: 0.857
Recall   : 1.000
